# Moth - Mura
MUNI Omniscient Tutor Helper - Masaryk University Repository Analyzer

This tool was created as a part of the thesis *"Measuring Software Development Contributions using Git"* thesis at Masaryk University.
The goal of this tool is to analyze git repositories of students and provide useful information to tutor about their work.

The implementation is originally written in Python 3.9 and uses the following libraries without which the tool would not be possible:
- [levenshtein](https://pypi.org/project/python-Levenshtein/) - for fuzzy matching in syntactic analysis
- [GitPython](https://gitpython.readthedocs.io/en/stable/) - for git operations
- [python-gitlab](https://python-gitlab.readthedocs.io/en/stable/) - for interfacing with GitLab
- [PyGithub](https://pygithub.readthedocs.io/en/latest/) - for interfacing with GitHub
- [matplotlib](https://matplotlib.org/) - for plotting various graphs
- [notebook](https://jupyter.org/) - for the front-end you are currently using

Below are the necessary imports for the tool to work.

In [ ]:
import fs_access as file_system
import lib
import mura
import configuration
import semantic_analysis

from uni_chars import *  # shortcut for unicode characters used throughout the tool
from history_analyzer import CommitRange

from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))  # wide screen support

# macros for automagically reloading the modules when they are changed
%load_ext autoreload
%autoreload 2

print(f"{SUCCESS} Imports loaded successfully.")

<a id='toc'></a>
# Table of contents
## Setup
- [Configuration](#configuration)
- [Repository setup](#repository)
- [Contributors](#contributors)
- [Config Overrides](#overrides)
- [Analysis](#analysis)
## Results
- [Commits](#commits)
- [Commit Graph](#commit-graph)
- [File statistics](#file-stats)
- [File ownership](#percentage-ownership)
- [File ownership dir-tree](#ownership-dir-tree)
- [Line distribution](#lines)
- [Unmerged commits](#unmerged-commits)
- [Rules](#rules)
- [Syntax](#syntax)
- [Semantics](#semantics)
- [Constructs](#constructs)
- [Hour estimation](#hour-estimation)
- [Remote repository](#remote-repo)
- [Summary](#summary)

<a id='configuration'></a>
# Configuration [↩](#toc)

The following code block is a shortcut for opening the configuration folder. The configuration folder contains configuration.txt with general variables and a rules.txt which holds rules to use during ownership analysis.


In [ ]:
# configuration.open_configuration_folder()


Apart from the two files mentioned before, separate folders exist for `lang-syntax` and `lang-semantics`. Each definition of a language is stored in a file/folder matching the language's file extension.
The `lang-syntax` folder contains weight definitions for the language's syntax.
The `lang-semantics` folder contains weight definitions for semantics and semantic analyzers themselves as an executable and a launch command for interfacing with the driver python code.
A separate folder `remote-repo-weights` contains weight definitions for remote repository objects: Issues and Pull Requests.

## Configuration file
- `configuration_data/configuration.txt` - contains general configuration of the tool
- `lang-syntax/*` - contains weight definitions for general syntax of a language
- `lang-semantics/*` - contains weight definitions for semantic constructs
- `remote-repo-weigths/weights.txt` - contains weight definitions for remote repository objects

## Rules file

- `configuration_data/rules.txt` - contains rules for the tool

Once the configuration is set, run the code block below to load the configurations into the tool.

In [ ]:
config = configuration.validate()

# All properties of the config can be edited here. An editor with code completion support is recommended.
# config.ignore_remote_repo = True

# Add extensions to the list to ignore during analysis, provide extensions will not be analyzer even if an analyzer is present.
config.ignored_extensions = ['.cs']

Semantic analyzers for each extension are executed only if the extension is present in the repository. Therefore, it is not required to have all the prerequisites installed if the project does not include those extensions.

<a id='repository'></a>
# Repository [↩](#toc)

Put the path to the repository you want to analyze into the `repository_path` variable and run the code block below.

In [ ]:
repository_path = r"/path/to/the/repository/directory"

repository = file_system.validate_repository(repository_path, config)

<a id='commit-range'></a>
# Commit range [↩](#toc)
The commit range is defined by the `start` and `end` variables. The variables can be either a commit hash or a tag/branch name.
Additionally, the `end` variable can be set to `ROOT` and `start` to `HEAD` to analyze the repository from the beginning to the current state.

In [ ]:
start = "HEAD"
end = "ROOT"

commit_range = CommitRange(repository, start, end, verbose=True)

# The expected amount of hours students are expected to spend on the project, used for hour weight estimation.
hour_estimate_per_contributor = 24

<a id='contributors'></a>
## Contributors [↩](#toc)
Displays a table of all contributors in the repository. The table contains the following information:
- Name
- Email
- Aliases

Aliases are put in place to group contributors who do not have a synchronized git configuration across all development devices. The aliases are used to match the contributors. If an alias matches commit author, the primary contributor name is used instead. Primary in this case meaning the first encounter.

In [ ]:
raw_contributors = mura.display_contributor_info(commit_range, config)

## Contributors

Often times contributors do not have a synchronized git configuration across all development devices. This can lead to the tool not being able to properly group contributions to the correct contributor. The tool attempts to match contributors by their name and email. If that is not enough an explicit name-to-name mapping can be provided in the `contributor_map` variable.

In [ ]:
contributor_map = \
[
    # ('Jiří Šťastný', 'Jiri Stastny'),
]

config.contributor_map = contributor_map

contributors = mura.display_contributor_info(commit_range, config)

<a id='overrides'></a>
## Overrides [↩](#toc)
In case merge conflicts occur, the file ownership is contributed towards the author of the resolving commit. This can lead to incorrect ownership attribution. To fix this, the ownership can be overridden by the following code block. The key is the complete commit hash and the value is the contributor name.

In [ ]:
commit_range.ownership_overrides["commit_hexsha"] = "Jiri Stastny"

# Uncomment the following line to enable anonymous mode, which will replace the names of the contributors with "Contributor #n"
# config.anonymous_mode = True

# Uncomment the following line to force skip analysis of a specific extension, e.g. ".cs".
# config.ignored_extensions = [".cs"]


<a id='analysis'></a>
# Analysis [↩](#toc)
The analysis is a time-consuming process. Taking longer the larger the repository is. For a single project from the PA165 course for Milestone 1 (120 commits), the analysis took about 35 seconds on an Intel i7-12700H CPU.

In [ ]:
tracked_files = lib.get_tracked_files(repository, verbose=True)
history_analysis_result = commit_range.analyze(verbose=True)
semantic_analysis_grouped_result = semantic_analysis.compute_semantic_weight_result(config, tracked_files, verbose=True)

# Results

The analysis part is finished. The tool provides multiple outputs to help the tutor analyze the students' work. Each output is a separate function code block. Seach section links back to the Table of Contents to make it easier to navigate.

<a id='commits'></a>
## Commits [↩](#toc)

In [ ]:
commit_distribution, insertions_deletions = mura.commit_info(commit_range, repository, contributors)

In [ ]:
mura.insertions_deletions_info(insertions_deletions)

<a id='commit-graph'></a>
## Commit graph [↩](#toc)
Displays a graph of the commits in the repository. The range of the x-axis is computed from the starting commit date and the ending commit date.
To display only a section of the graph, the list can be sliced. This is generally useful when there are commits at the boundaries. Taking a section in the middle does not make much sense.

In [ ]:
commits = [commit for commit in commit_range]

commits = commits[1:]  # remove first commit
# commits = commits[:10] # remove last 10 commits

mura.plot_commits(commits, commit_range, contributors, repository, force_x_axis_labels=False)

<a id='file-stats'></a>
## File statistics [↩](#toc)
First part of the output is a combined statistics of all file changes in the repository.

- A: Files Added
- D: Files Deleted
- M: Files Modified
- R: Files Renamed

The statistics are cumulative. Meaning if a file is added and in any subsequent commit it is deleted, the file is counted towards both statistics.

In [ ]:
flagged_files = mura.file_statistics_info(commit_range, contributors)

<a id='percentage-ownership'></a>
## Percentages and ownership [↩](#toc)

In [ ]:
percentage, ownership = mura.percentage_info(history_analysis_result, contributors, config)

<a id='ownership-dir-tree'></a>
## Ownership as a directory tree [↩](#toc)

In [ ]:
mura.display_dir_tree(config, percentage, repository)

<a id='lines'></a>
## Lines, Blanks and Comments [↩](#toc)

In [ ]:
mura.lines_blanks_comments_info(repository, ownership, semantic_analysis_grouped_result, tracked_files, contributors)

<a id='unmerged-commits'></a>
## Unmerged commits [↩](#toc)

In [ ]:
commit_range.unmerged_commits_info(repository, config, contributors)

<a id='rules'></a>
## Rules [↩](#toc)

In [ ]:
rule_violation_weight_multipliers = mura.rule_info(config, repository, ownership, contributors)

<a id='syntax'></a>
## Syntax [↩](#toc)

In [ ]:
syntactic_weights = mura.syntax_info()

<a id='semantics'></a>
## Semantics [↩](#toc)

In [ ]:
semantic_weights = mura.semantic_info(tracked_files, ownership, semantic_analysis_grouped_result)

<a id='constructs'></a>
## Constructs and ownership [↩](#toc)

In [ ]:
mura.constructs_info(tracked_files, ownership, semantic_analysis_grouped_result)

<a id='hour-estimation'></a>
## Hour estimation [↩](#toc)

In [ ]:
hour_estimates = mura.hour_estimates(contributors, repository)

hour_weights = mura.gaussian_weights(config, hour_estimate_per_contributor, hour_estimates)

<a id='remote-repo'></a>
## Remote repository [↩](#toc)

In [ ]:
repo_management_weights = mura.remote_info(commit_range, repository, config, contributors)

<a id='summary'></a>
## Summary [↩](#toc)

In [ ]:
mura.summary_info(contributors, syntactic_weights, semantic_weights, repo_management_weights, rule_violation_weight_multipliers, hour_weights)